##Importing Libraries

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from scipy import stats
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import FitFailedWarning


## Importing Data

In [3]:
data = pd.read_csv('wineq.csv')
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Algorithm Selection

In [4]:

models = [
    ('Random Forest', RandomForestClassifier()),
    ('SVM', SVC()),
    ('Logistic Regression', LogisticRegression()),
    ('Decision Tree', DecisionTreeClassifier())
]


## Cross-Validation and Model Evaluation

In [5]:
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
results = {}
for name, model in models:
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    results[name] = scores

## Final Selection

In [6]:
final_scores = {}
for name, model in models:
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    final_scores[name] = accuracy

best_model = max(final_scores, key=final_scores.get)
for name, accuracy in final_scores.items():
    print(f"{name} Accuracy: {accuracy}")
print("Best Model:", best_model, final_scores[best_model])



Random Forest Accuracy: 0.71875
SVM Accuracy: 0.503125
Logistic Regression Accuracy: 0.63125
Decision Tree Accuracy: 0.678125
Best Model: Random Forest 0.71875


## Hyperparameter Optimization

In [7]:
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold

classifiers = {
    'Logistic Regression': LogisticRegression(max_iter=100),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC()
}

param_grids = {
    'Logistic Regression': {'C': [0.1, 0.5, 1, 5, 10], 'penalty': ['l1', 'l2'], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
    'Decision Tree': {'max_depth': [None, 10, 20, 30, 50, 100], 'min_samples_split': [2, 5, 10, 15], 'min_samples_leaf': [1, 2, 4, 7]},
    'Random Forest': {'n_estimators': [10, 50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
    'SVM': {'C': [0.1, 0.8, 2, 10], 'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}
}


for clf_name, clf in classifiers.items():
    param_grid = param_grids[clf_name]
    grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    cv_scores = cross_val_score(grid_search, X_train, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))

    best_model = grid_search.fit(X_train, y_train).best_estimator_
    test_accuracy = best_model.score(X_test, y_test)

    print(f"Nested Cross-Validation Results for {clf_name}:")
    print("Best Hyperparameters:", best_model.get_params())
    print("Cross-Validation Accuracy: {:.4f} (+/- {:.4f})".format(cv_scores.mean(), cv_scores.std() * 2))
    print("Test Accuracy: {:.4f}".format(test_accuracy))
    print("\n")
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold

classifiers = {
    'Logistic Regression': LogisticRegression(max_iter=100),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC()
}


param_grids = {
    'Logistic Regression': {'C': [0.1, 0.5, 1, 5, 10], 'penalty': ['l1', 'l2'], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
    'Decision Tree': {'max_depth': [None, 10, 20, 30, 50, 100], 'min_samples_split': [2, 5, 10, 15], 'min_samples_leaf': [1, 2, 4, 7]},
    'Random Forest': {'n_estimators': [10, 50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
    'SVM': {'C': [0.1, 0.8, 2, 10], 'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}
}

# Perform nested resampling
for clf_name, clf in classifiers.items():
    param_grid = param_grids[clf_name]
    grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    cv_scores = cross_val_score(grid_search, X_train, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))

    best_model = grid_search.fit(X_train, y_train).best_estimator_
    test_accuracy = best_model.score(X_test, y_test)

    print(f"Nested Cross-Validation Results for {clf_name}:")
    print("Best Hyperparameters:", best_model.get_params())
    print("Cross-Validation Accuracy: {:.4f} (+/- {:.4f})".format(cv_scores.mean(), cv_scores.std() * 2))
    print("Test Accuracy: {:.4f}".format(test_accuracy))
    print("\n")


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
75 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solv

Nested Cross-Validation Results for Logistic Regression:
Best Hyperparameters: {'C': 0.5, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Cross-Validation Accuracy: 0.5833 (+/- 0.0254)
Test Accuracy: 0.6250


Nested Cross-Validation Results for Decision Tree:
Best Hyperparameters: {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 50, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}
Cross-Validation Accuracy: 0.5786 (+/- 0.0500)
Test Accuracy: 0.6500


Nested Cross-Validation Results for Random Forest:
Best Hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'g

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
75 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solv

Nested Cross-Validation Results for Logistic Regression:
Best Hyperparameters: {'C': 0.5, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Cross-Validation Accuracy: 0.5833 (+/- 0.0254)
Test Accuracy: 0.6250


Nested Cross-Validation Results for Decision Tree:
Best Hyperparameters: {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 30, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}
Cross-Validation Accuracy: 0.5715 (+/- 0.0085)
Test Accuracy: 0.6625


Nested Cross-Validation Results for Random Forest:
Best Hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'g

## Final Selection after GridSeach



In [ ]:

final_scores = {
    'Random Forest': rf_accuracy,
    'SVM': svm_accuracy,
    'Logistic Regression': logreg_accuracy,
    'Decision Tree': dt_accuracy
}

best_model = max(final_scores, key=final_scores.get)

for name, accuracy in final_scores.items():
    print(f"{name} Accuracy: {accuracy}")

print("Best Model:", best_model, final_scores[best_model])


Random Forest Accuracy: 0.703125
SVM Accuracy: 0.6375
Logistic Regression Accuracy: 0.625
Decision Tree Accuracy: 0.665625
Best Model: Random Forest 0.703125


In [ ]:
!pip install scikit-optimize
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

data = pd.read_csv('wineq.csv')
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Outer loop for nested cross-validation
outer_cv = 8
inner_cv = 8
outer_scores = {}

# Algorithm Selection
models = [
    ('SVM', SVC()),
    ('Random Forest', RandomForestClassifier()),
    ('Logistic Regression', LogisticRegression()),
    ('Decision Tree', DecisionTreeClassifier())
]

# Hyperparameter Optimization using Bayesian Optimization
param_grids = {
    'SVM': {
         'C': Real(1e-5, 1e+5, prior='log-uniform'),
        'kernel': Categorical(['linear', 'rbf', 'poly', 'sigmoid'])
    },
    'Random Forest': {
        'n_estimators': Integer(10, 200),
        'max_depth': Integer(10, 30),
        'min_samples_split': Integer(1, 10),
        'min_samples_leaf': Integer(1, 10)
    },
    'Logistic Regression': {
        'C': Real(1e-6, 1e+6, prior='log-uniform'),
        'penalty': Categorical(['l1', 'l2']),
        'solver': Categorical(['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])
    },
    'Decision Tree': {
        'max_depth': Integer(1, 100),
        'min_samples_split': Integer(2, 15),
        'min_samples_leaf': Integer(1, 7)
    }
}

for name, model in models:
    try:
        outer_scores[name] = []
        for outer_train_index, outer_test_index in StratifiedKFold(n_splits=outer_cv, shuffle=True, random_state=0).split(X, y):
            X_outer_train, X_outer_test = X[outer_train_index], X[outer_test_index]
            y_outer_train, y_outer_test = y[outer_train_index], y[outer_test_index]

            # Inner loop for hyperparameter tuning
            opt = BayesSearchCV(
                model,
                param_grids[name],
                n_iter=10,
                cv=StratifiedKFold(n_splits=inner_cv, shuffle=True, random_state=0),
                scoring='accuracy',
                n_jobs=-1,
                random_state=0
            )
            opt.fit(X_outer_train, y_outer_train)
            best_model = opt.best_estimator_

            # Evaluate on the outer test set
            accuracy = best_model.score(X_outer_test, y_outer_test)
            outer_scores[name].append(accuracy)

            print(f"{name} - Tuned Hyperparameters: {opt.best_params_}, Accuracy: {accuracy}")

    except Exception as e:
        print(f"Bayesian Optimization for {name} raised an exception: {e}")

# Display results
for name, scores in outer_scores.items():
    mean_accuracy = np.nanmean(scores)  # Handling NaN values
    print(f"{name} Outer CV Mean Accuracy: {mean_accuracy}")

best_outer_model_name = max(outer_scores, key=outer_scores.get)
print("Best Outer Model:", best_outer_model_name, np.mean(outer_scores[best_outer_model_name]))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 1.3 MB/s eta 0:00:00
SVM - Tuned Hyperparameters: OrderedDict([('C', 46915.05705722399), ('kernel', 'rbf')]), Accuracy: 0.635
SVM - Tuned Hyperparameters: OrderedDict([('C', 1.9360537965638225), ('kernel', 'linear')]), Accuracy: 0.59
SVM - Tuned Hyperparameters: OrderedDict([('C', 46915.05705722399), ('kernel', 'rbf')]), Accuracy: 0.66
SVM - Tuned Hyperparameters: OrderedDict([('C', 46915.05705722399), ('kernel', 'rbf')]), Accuracy: 0.61
SVM - Tuned Hyperparameters: OrderedDict([('C', 46915.05705722399), ('kernel', 'rbf')]), Accuracy: 0.6
SVM - Tuned Hyperparameters: OrderedDict([('C', 46915.05705722399), ('kernel', 'rbf')]), Accuracy: 0.54
SVM - Tuned Hyperparameters: OrderedDict([('C', 46915.05705722399), ('kernel', 'rbf')]), Accuracy: 0.595
SVM - Tuned Hyperparameters: OrderedDict([('C', 46915.05705722399), ('kernel', 'rbf')]), Accuracy: 0.5829145728643216
Random Forest - Tuned Hyperparameters: OrderedDict([('max_depth', 2

<ipython-input-9-8633489edd76>:84: RuntimeWarning: Mean of empty slice
  mean_accuracy = np.nanmean(scores)  # Handling NaN values


In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import numpy as np

model_results = [outer_scores[model_name] for model_name in outer_scores]

flat_results = [score for scores in model_results for score in scores]


labels = []
for model_name, scores in outer_scores.items():
    labels.extend([model_name] * len(scores))


tukey_results = pairwise_tukeyhsd(np.array(flat_results), labels, alpha=0.05)
from tabulate import tabulate
tukey_df = pd.DataFrame(data=tukey_results._results_table.data[1:], columns=tukey_results._results_table.data[0])
print(tabulate(tukey_df, headers='keys', tablefmt='pretty', showindex=False))

+---------------+---------------+----------+--------+---------+---------+--------+
|    group1     |    group2     | meandiff | p-adj  |  lower  |  upper  | reject |
+---------------+---------------+----------+--------+---------+---------+--------+
| Decision Tree | Random Forest |  0.0888  | 0.0007 | 0.0382  | 0.1394  |  True  |
| Decision Tree |      SVM      | -0.0069  | 0.9373 | -0.0575 | 0.0437  | False  |
| Random Forest |      SVM      | -0.0957  | 0.0003 | -0.1463 | -0.0451 |  True  |
+---------------+---------------+----------+--------+---------+---------+--------+
